In [1]:
import pandas as pd

import surprise
from surprise.prediction_algorithms import *
import pandas as pd
import numpy as np
import datetime as dt

from surprise import Dataset
from surprise import Reader

from surprise.prediction_algorithms import knns
from surprise.similarities import cosine, msd, pearson
from surprise import accuracy
from surprise.model_selection import GridSearchCV

In [5]:
movies = pd.read_csv('../../ml-latest-small/movies.csv')

In [6]:
link = pd.read_csv('../../ml-latest-small/links.csv')

In [7]:
rating = pd.read_csv('../../ml-latest-small/ratings.csv')

In [8]:
tags = pd.read_csv('../../ml-latest-small/tags.csv')

In [9]:
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [10]:
link

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0
...,...,...,...
9737,193581,5476944,432131.0
9738,193583,5914996,445030.0
9739,193585,6397426,479308.0
9740,193587,8391976,483455.0


In [11]:
rating

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [12]:
tags

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200
...,...,...,...,...
3678,606,7382,for katie,1171234019
3679,606,7936,austere,1173392334
3680,610,3265,gun fu,1493843984
3681,610,3265,heroic bloodshed,1493843978


The DF tags has 'Tag" descriptions of moves provided by users. 

In [25]:
movie_rating = movies.merge(rating, on='movieId', how='outer')

In [28]:
movie_rating

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1.0,4.0,9.649827e+08
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5.0,4.0,8.474350e+08
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7.0,4.5,1.106636e+09
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15.0,2.5,1.510578e+09
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17.0,4.5,1.305696e+09
...,...,...,...,...,...,...
100849,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy,184.0,4.0,1.537109e+09
100850,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy,184.0,3.5,1.537110e+09
100851,193585,Flint (2017),Drama,184.0,3.5,1.537110e+09
100852,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation,184.0,3.5,1.537110e+09


In [29]:
movie_rating.isna().sum()

movieId       0
title         0
genres        0
userId       18
rating       18
timestamp    18
dtype: int64

In [30]:
movie_rating.dropna(inplace=True)

In [31]:
movie_rating.shape

(100836, 6)

In [32]:
movie_rating['userId'].nunique()

610

In [33]:
stats = movie_rating[['rating', 'timestamp']].describe()
stats

,rating,timestamp
count,100836.000000,1.008360e+05
mean,3.501557,1.205946e+09
std,1.042529,2.162610e+08
min,0.500000,8.281246e+08
25%,3.000000,1.019124e+09
50%,3.500000,1.186087e+09
75%,4.000000,1.435994e+09
max,5.000000,1.537799e+09


In [34]:
print(dt.datetime.fromtimestamp(stats.loc['min', 'timestamp']))
print(dt.datetime.fromtimestamp(stats.loc['max', 'timestamp']))

1996-03-29 12:36:55
2018-09-24 09:27:30


## Split

In [76]:
from sklearn.model_selection import train_test_split

In [77]:
train, test = train_test_split(movie_rating)

In [78]:
train

,movieId,title,genres,userId,rating,timestamp
46574,2692,Run Lola Run (Lola rennt) (1998),Action|Crime,354.0,4.5,1.200871e+09
11088,376,"River Wild, The (1994)",Action|Thriller,150.0,3.0,8.542031e+08
75922,8363,Soul Plane (2004),Comedy,484.0,4.5,1.342369e+09
5458,190,Safe (1995),Thriller,195.0,1.0,9.747103e+08
3713,111,Taxi Driver (1976),Crime|Drama|Thriller,489.0,3.0,1.333102e+09
...,...,...,...,...,...,...
61894,4367,Lara Croft: Tomb Raider (2001),Action|Adventure,200.0,2.5,1.229888e+09
48182,2795,National Lampoon's Vacation (1983),Comedy,274.0,3.5,1.249768e+09
15229,585,"Brady Bunch Movie, The (1995)",Comedy,474.0,3.0,1.060106e+09
2298,47,Seven (a.k.a. Se7en) (1995),Mystery|Thriller,580.0,5.0,1.167791e+09


In [79]:
reader = Reader(rating_scale=(1, 5))
train_data = Dataset.load_from_df(train[['userId', 'movieId', 'rating']], reader)

In [37]:
data

In [63]:
train, test = train_test_split(data, random_state=42)

In [64]:
train

In [66]:
print('Number of users: ', train.n_users, '\n')
print('Number of items: ', train.n_items, '\n')

Number of users:  610 

Number of items:  9007 



^ We want to use user based.

In [67]:
model1 = KNNBasic()

In [68]:
model2 = SVD()

In [69]:
model3 = NMF()

In [70]:
model4 = knns.KNNWithMeans()

## Tuning

In [88]:
param_grid = {'k':[10, 50, 100],'min_k': [1, 5, 10]}
base_model = GridSearchCV(KNNBasic,param_grid=param_grid,joblib_verbose=5)
base_model.fit(train_data)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Computing the msd similarity matrix...
Done computing similarity matrix.


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.4s remaining:    0.0s


Computing the msd similarity matrix...
Done computing similarity matrix.


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.9s remaining:    0.0s


...

[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    1.3s remaining:    0.0s


Computing the msd similarity matrix...
Done computing similarity matrix.


[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    1.8s remaining:    0.0s


Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computi

[Parallel(n_jobs=1)]: Done  45 out of  45 | elapsed:   25.7s finished


In [92]:
base_model.best_params

{'rmse': {'k': 10, 'min_k': 5}, 'mae': {'k': 10, 'min_k': 5}}

In [ ]:
# param_grid = {'n_factors':[20, 100],'n_epochs': [5, 10], 'lr_all': [0.002, 0.005],
#               'reg_all': [0.4, 0.6]}
# gs_model = GridSearchCV(SVD,param_grid=param_grid,n_jobs = -1,joblib_verbose=5)
# gs_model.fit(jokes)

## Cross Validation

In [83]:
cv1 = surprise.model_selection.split.KFold()

In [84]:
cv1_split = cv1.split(train_data)

In [85]:
cv1_split

<generator object KFold.split at 0x7ff460615740>

In [86]:
from surprise.model_selection import cross_validate

In [87]:
cross_validate(model, train_data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9694  0.9670  0.9603  0.9622  0.9640  0.9646  0.0033  
MAE (testset)     0.7440  0.7456  0.7367  0.7387  0.7356  0.7401  0.0040  
Fit time          0.04    0.04    0.04    0.04    0.04    0.04    0.00    
Test time         0.55    0.55    0.54    0.53    0.54    0.54    0.01    


{'test_rmse': array([0.96942726, 0.96696192, 0.96031022, 0.96218592, 0.96396175]),
 'test_mae': array([0.74403068, 0.74562693, 0.73673629, 0.73872971, 0.7356187 ]),
 'fit_time': (0.03617405891418457,
  0.044302940368652344,
  0.03999614715576172,
  0.03700900077819824,
  0.04159116744995117),
 'test_time': (0.5473847389221191,
  0.5497086048126221,
  0.5435881614685059,
  0.5336771011352539,
  0.5417799949645996)}